In [5]:
pip install opencv-python mediapipe numpy


Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp
import numpy as np


In [3]:
!pip install opencv-python mediapipe numpy


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)

# Color selection
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255), (255, 0, 255), (255, 255, 0), (0, 0, 0)]  # Added Black for better contrast
color_index = 0  # Default color: Blue

# Shape selection
shapes = ['Freehand', 'Rectangle', 'Circle']
shape_index = 0  # Default shape: Freehand
start_x, start_y = None, None  # For shape drawing

# Brush thickness
thickness = 5

# Canvas for drawing
canvas = np.zeros((480, 640, 3), dtype=np.uint8)
canvas.fill(255)  # Make the canvas white for visibility
prev_x, prev_y = None, None  # Store previous position for continuous drawing

# Capture video from webcam
cap = cv2.VideoCapture(0)

def draw_color_palette(frame):
    for i, color in enumerate(colors):
        x_start = i * 50
        cv2.rectangle(frame, (x_start, 0), (x_start + 50, 50), color, -1)
        if i == color_index:
            cv2.rectangle(frame, (x_start, 0), (x_start + 50, 50), (0, 0, 0), 3)  # Make selection outline darker

def draw_shape_box(frame):
    for i, shape in enumerate(shapes):
        x_start = 500 + i * 50
        cv2.rectangle(frame, (x_start, 420), (x_start + 50, 470), (100, 100, 100), -1)  # Darker background
        cv2.putText(frame, shape[:3], (x_start + 5, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)  # White text
        if i == shape_index:
            cv2.rectangle(frame, (x_start, 420), (x_start + 50, 470), (0, 0, 0), 3)  # Make selection outline darker

def is_hand_closed(hand_landmarks):
    fingers = [8, 12, 16, 20]  # Index, Middle, Ring, Pinky tips
    for finger in fingers:
        if hand_landmarks.landmark[finger].y < hand_landmarks.landmark[finger - 2].y:
            return False  # If any finger is up, hand is not closed
    return True  # Hand is closed

def are_index_middle_fingers_up(hand_landmarks):
    return (hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y and  
            hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y)  # Index & Middle fingers up

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    h, w, c = frame.shape
    
    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process hand tracking
    result = hands.process(rgb_frame)
    
    # Draw color palette
    draw_color_palette(frame)
    draw_shape_box(frame)
    
    # Check if hand is detected
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                      mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3), 
                                      mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))  # Make hand landmarks clearer
            
            if is_hand_closed(hand_landmarks):
                canvas.fill(255)  # Clear the canvas if hand is closed
                prev_x, prev_y = None, None
                
            index_finger = hand_landmarks.landmark[8]  # Tip of index finger
            x, y = int(index_finger.x * w), int(index_finger.y * h)
            
            # Check if selecting a color
            if y < 60:
                color_index = x // 50 if x // 50 < len(colors) else color_index
                prev_x, prev_y = None, None  # Reset previous position
            
            # Check if selecting a shape
            elif 420 < y < 470 and 500 < x < 650:
                shape_index = (x - 500) // 50 if (x - 500) // 50 < len(shapes) else shape_index
            
            # Stop drawing if both index and middle fingers are up
            elif are_index_middle_fingers_up(hand_landmarks):
                prev_x, prev_y = None, None  # Stop drawing
            
            else:
                # Drawing logic based on shape
                if shape_index == 0:  # Freehand drawing
                    if prev_x is not None and prev_y is not None:
                        cv2.line(canvas, (prev_x, prev_y), (x, y), colors[color_index], thickness)
                    prev_x, prev_y = x, y  # Update previous position
                elif shape_index == 1:  # Rectangle
                    if start_x is None and start_y is None:
                        start_x, start_y = x, y
                    else:
                        cv2.rectangle(canvas, (start_x, start_y), (x, y), colors[color_index], thickness)
                        start_x, start_y = None, None
                elif shape_index == 2:  # Circle
                    if start_x is None and start_y is None:
                        start_x, start_y = x, y
                    else:
                        radius = int(((x - start_x) ** 2 + (y - start_y) ** 2) ** 0.5)
                        cv2.circle(canvas, (start_x, start_y), radius, colors[color_index], thickness)
                        start_x, start_y = None, None
    
    # Combine the canvas with the original frame to show tools and hands
    frame = cv2.addWeighted(frame, 0.6, canvas, 0.7, 0)
    
    # Show the output
    cv2.imshow("Virtual Painter", frame)
    
    # Keyboard controls
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Quit
        break
    elif key == ord('c'):  # Clear canvas manually
        canvas.fill(255)
        prev_x, prev_y = None, None  # Reset previous position

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)  # Ensures proper closure in Jupyter Notebook
